Download data from API


In [2]:
import pandas as pd
import requests

In [4]:
def get_player_data(player,season,game_type,team):
    player_data = { 'id' : player['person']['id'],
                    'player': player['person']['fullName'],
                    'position' : player['position']['name'],
                    'team' : team}
    if player['position']['code'] == 'G': return player_data
    player_id = player['person']['id']
    season_input = str(season)+str(season+1)
    
    #if the game_type requested is playoffs (P):
    if game_type == 'P': url = f"https://statsapi.web.nhl.com/api/v1/people/{player_id}/stats?stats=statsSingleSeasonPlayoffs&season={season_input}"
    else: url = f"https://statsapi.web.nhl.com/api/v1/people/{player_id}/stats?stats=statsSingleSeason&season={season_input}"
    
    #requesting data
    response = requests.get(url)
    player_stats = response.json()
    player_stats = player_stats['stats'][0]['splits'] #['stat']
    if len(player_stats) == 0: return player_data
    
    #different stats to display
    player_stats = player_stats[0]['stat']
    stats = {
            **player_data,
            'time on ice' : player_stats['timeOnIce'],
            'assists' : player_stats.get('assists'),
            'goals' : player_stats['goals'],
            'points' : player_stats['points'],
            'pim' : player_stats['pim'],
            'shots' : player_stats['shots'],
            'games' : player_stats['games'],
            'hits' : player_stats['hits'],
            'blocked shots' : player_stats['blocked'],
            'faceoff pct' : player_stats['faceOffPct'],
            'plus minus' : player_stats['plusMinus'] }
    return stats

def get_season_player_data(season,game_type):
    season_input = str(season)+str(season+1)
    url = "https://statsapi.web.nhl.com/api/v1/teams"
    response = requests.get(url)
    teams = response.json()
    team_ids = []
    all_players = []
    for team in teams['teams']:
        team_ids.append([team['id'],team['name']])
    for team_id,team_name in team_ids:
        url = f"https://statsapi.web.nhl.com/api/v1/teams/{team_id}/roster"
        response = requests.get(url)
        roster = response.json()
        for player in roster['roster']:
            all_players.append(get_player_data(player,season,game_type,team_name))
    return all_players
x = get_season_player_data(2022,'R')


In [5]:
df = pd.DataFrame(x)
#only take minutes from time on ice column
df['time on ice'] = df['time on ice'].str.slice(0,-3) #.astype(int)
df = df.dropna()
df['time on ice'] = df['time on ice'].astype(int)

df.sort_values('plus minus',ascending=False).head(10)

,id,player,position,team,time on ice,assists,goals,points,pim,shots,games,hits,blocked shots,faceoff pct,plus minus
131,8476854,Hampus Lindholm,Defenseman,Boston Bruins,1834,43.0,10.0,53.0,56.0,157.0,79.0,78.0,98.0,0.00,50.0
132,8476891,Matt Grzelcyk,Defenseman,Boston Bruins,1275,22.0,4.0,26.0,28.0,101.0,74.0,50.0,78.0,0.00,47.0
142,8478443,Brandon Carlo,Defenseman,Boston Bruins,1395,13.0,3.0,16.0,38.0,94.0,74.0,127.0,115.0,0.00,44.0
613,8470794,Joe Pavelski,Center,Dallas Stars,1435,49.0,28.0,77.0,8.0,182.0,81.0,91.0,64.0,53.30,42.0
2,8475193,Tomas Tatar,Left Wing,New Jersey Devils,1223,27.0,20.0,47.0,28.0,153.0,81.0,64.0,30.0,39.53,41.0
516,8478038,Devon Toews,Defenseman,Colorado Avalanche,1959,41.0,6.0,47.0,24.0,160.0,78.0,91.0,136.0,0.00,38.0
629,8480027,Jason Robertson,Left Wing,Dallas Stars,1525,63.0,46.0,109.0,20.0,309.0,81.0,57.0,19.0,0.00,37.0
138,8477956,David Pastrnak,Right Wing,Boston Bruins,1586,51.0,60.0,111.0,38.0,404.0,81.0,91.0,33.0,42.11,35.0
18,8480002,Nico Hischier,Center,New Jersey Devils,1542,48.0,31.0,79.0,10.0,253.0,80.0,33.0,61.0,53.55,34.0
125,8470638,Patrice Bergeron,Center,Boston Bruins,1350,31.0,27.0,58.0,22.0,249.0,77.0,66.0,54.0,60.99,34.0


In [171]:
df.to_csv('player_season_stats.csv')